Intro to Pydantic V2


Basic Model

In [1879]:
# pip install pydantic

In [1880]:
from pydantic import BaseModel

class Person(BaseModel):
    name: str
    age: int

In [1881]:
p = Person(name='John Doe', age=30)
print(p)

name='John Doe' age=30


In [1882]:
from pydantic import ValidationError

try:
    p = Person(name='John Doe', age='junk')
except ValidationError as e:
    print(e)

1 validation error for Person
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='junk', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/int_parsing


Validation Exception

In [1883]:
try:
    Person(name=100, age='junk')
except ValidationError as e:
    exception = e

In [1884]:
exception.errors()

[{'type': 'string_type',
  'loc': ('name',),
  'msg': 'Input should be a valid string',
  'input': 100,
  'url': 'https://errors.pydantic.dev/2.10/v/string_type'},
 {'type': 'int_parsing',
  'loc': ('age',),
  'msg': 'Input should be a valid integer, unable to parse string as an integer',
  'input': 'junk',
  'url': 'https://errors.pydantic.dev/2.10/v/int_parsing'}]

In [1885]:
exception.json()

'[{"type":"string_type","loc":["name"],"msg":"Input should be a valid string","input":100,"url":"https://errors.pydantic.dev/2.10/v/string_type"},{"type":"int_parsing","loc":["age"],"msg":"Input should be a valid integer, unable to parse string as an integer","input":"junk","url":"https://errors.pydantic.dev/2.10/v/int_parsing"}]'

Deserialization of Data

In [1886]:
data = {'name': 'John Doe', 'age': 30}

In [1887]:
p = Person.model_validate(data)
p

Person(name='John Doe', age=30)

In [1888]:
data_json = '{"name": "John Doe", "age": 20}'

p = Person.model_validate_json(data_json)
p

Person(name='John Doe', age=20)

Required vs Optional Fields

In [1889]:
try:
    Person(age=29)
except ValidationError as e:
    print(e)

1 validation error for Person
name
  Field required [type=missing, input_value={'age': 29}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing


In [1890]:
data = {'name': 'John Doe'}

try:
    Person.model_validate(data)
except ValidationError as e:
    print(e)

1 validation error for Person
age
  Field required [type=missing, input_value={'name': 'John Doe'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing


In [1891]:
# Create a default value for a field
class Person(BaseModel):
    name: str
    age: int = 30

In [1892]:
Person.model_fields

{'name': FieldInfo(annotation=str, required=True),
 'age': FieldInfo(annotation=int, required=False, default=30)}

In [1893]:
p = Person(name='John Doe')
p

Person(name='John Doe', age=30)

Nullable Fields

In [1894]:
class Person(BaseModel):
    name: str | None = None
    age: int = 30

In [1895]:
Person.model_fields

{'name': FieldInfo(annotation=Union[str, NoneType], required=False, default=None),
 'age': FieldInfo(annotation=int, required=False, default=30)}

In [1896]:
p = Person()
p

Person(name=None, age=30)

In [1897]:
from typing import Union

class Person(BaseModel):
    name: Union[str, None] = None
    age: int = 30

In [1898]:
Person.model_fields

{'name': FieldInfo(annotation=Union[str, NoneType], required=False, default=None),
 'age': FieldInfo(annotation=int, required=False, default=30)}

In [1899]:
from typing import Optional

class Person(BaseModel):
    name: Optional[str] = None
    age: int = 30

In [1900]:
Person.model_fields

{'name': FieldInfo(annotation=Union[str, NoneType], required=False, default=None),
 'age': FieldInfo(annotation=int, required=False, default=30)}

List Fields

In [1901]:
class Person(BaseModel):
    name: str
    age: int
    lucky_numbers: list[int] = []

In [1902]:
Person.model_fields

{'name': FieldInfo(annotation=str, required=True),
 'age': FieldInfo(annotation=int, required=True),
 'lucky_numbers': FieldInfo(annotation=list[int], required=False, default=[])}

In [1903]:
p = Person(name='John Doe', age=30, lucky_numbers=[1, '2', 3.0])
p

Person(name='John Doe', age=30, lucky_numbers=[1, 2, 3])

In [1904]:
for number in p.lucky_numbers:
    print(number, type(number))

1 <class 'int'>
2 <class 'int'>
3 <class 'int'>


Aliases and Field Class


In [1905]:
data = {
    "id": 123,
    "First Name": "John",
    "LASTNAME": "Doe",
    "age in years": 30,
}

In [1906]:
from pydantic import Field

class Person(BaseModel):
    id_: int = Field(alias='id')
    first_name: str = Field(alias='First Name')
    last_name: str = Field(alias='LASTNAME')
    age: int = Field(alias='age in years')

p = Person.model_validate(data)
p

Person(id_=123, first_name='John', last_name='Doe', age=30)

In [1907]:
# deserialize
p.model_dump()

{'id_': 123, 'first_name': 'John', 'last_name': 'Doe', 'age': 30}

In [1908]:
p.model_dump_json()

'{"id_":123,"first_name":"John","last_name":"Doe","age":30}'

In [1909]:
p.model_dump(by_alias=True)

{'id': 123, 'First Name': 'John', 'LASTNAME': 'Doe', 'age in years': 30}

In [1910]:
p.model_dump_json(by_alias=True)

'{"id":123,"First Name":"John","LASTNAME":"Doe","age in years":30}'

In [1911]:
class Person(BaseModel):
    first_name: str | None = Field(alias='First Name', default=None)
    last_name: str | None = Field(alias='LASTNAME')

In [1912]:
data = {
    "LASTNAME": "Doe",
}

p = Person.model_validate(data)
p

Person(first_name=None, last_name='Doe')

In [1913]:
try:
    Person(first_name='John')
except ValidationError as e:
    print(e)

1 validation error for Person
LASTNAME
  Field required [type=missing, input_value={'first_name': 'John'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing


Model Config: Populate By Name

In [1914]:
from pydantic import ConfigDict

class Person(BaseModel):
    model_config = ConfigDict(populate_by_name=True)

    first_name: str | None = Field(alias='First Name', default=None)
    last_name: str = Field(alias='LASTNAME')

In [1915]:
p = Person(last_name='Doe')
p

Person(first_name=None, last_name='Doe')

In [1916]:
data = {
    "LASTNAME": "Doe",
    "first_name": "John",
}

p = Person.model_validate(data)
p

Person(first_name='John', last_name='Doe')

Mutable Defaults

In [1917]:
class LotteryTicket(BaseModel):
    numbers: list[int] = []

In [1918]:
t1 = LotteryTicket()
t2 = LotteryTicket()

In [1919]:
t1.numbers.extend([1, 2, 3])
t1

LotteryTicket(numbers=[1, 2, 3])

In [1920]:
t2.numbers

[]

Default Factories

In [1921]:
from datetime import datetime, timezone

class Log(BaseModel):
    timestamp: datetime = Field(default_factory= lambda: datetime.now(tz=timezone.utc))
    message: str

In [1922]:
log1 = Log(message='Hello')

log2 = Log(message='World')

In [1923]:
log1

Log(timestamp=datetime.datetime(2025, 1, 18, 23, 45, 26, 523202, tzinfo=datetime.timezone.utc), message='Hello')

In [1924]:
log2

Log(timestamp=datetime.datetime(2025, 1, 18, 23, 45, 26, 523257, tzinfo=datetime.timezone.utc), message='World')

Custom Serializers

In [1925]:
class Model(BaseModel):
    number: float

In [1926]:
m = Model(number=1.0)
m.model_dump_json()

'{"number":1.0}'

In [1927]:
m = Model(number=1/3)
m.model_dump_json()

'{"number":0.3333333333333333}'

In [1928]:
dt = datetime.now(timezone.utc)
dt.isoformat()

'2025-01-18T23:45:26.676896+00:00'

In [1929]:
class Model(BaseModel):
    dt: datetime

In [1930]:
m  =Model(dt= datetime.now(timezone.utc))
m

Model(dt=datetime.datetime(2025, 1, 18, 23, 45, 26, 698327, tzinfo=datetime.timezone.utc))

In [1931]:
m.model_dump_json()

'{"dt":"2025-01-18T23:45:26.698327Z"}'

In [1932]:
m.model_dump()

{'dt': datetime.datetime(2025, 1, 18, 23, 45, 26, 698327, tzinfo=datetime.timezone.utc)}

In [1933]:
from pydantic import field_serializer

In [1934]:
class Model(BaseModel):
    number: float
    
    @field_serializer('number')
    def serialize_number(self, v: float) -> float:
        return round(v, 2)

In [1935]:
m = Model(number=1/3)
m.model_dump()

{'number': 0.33}

In [1936]:
m.model_dump_json()

'{"number":0.33}'

In [1937]:
class Model(BaseModel):
    number: float
    dt :  datetime
    
    @field_serializer('number')
    def serialize_number(self, v: float) -> float:
        return round(v, 2)
    
    @field_serializer('dt', when_used="json-unless-none")
    def serialize_dt_to_json(self, v: datetime) -> str:
        return v.strftime('%Y-%m-%d')

In [1938]:
m = Model(number=1/3, dt=datetime.now(timezone.utc))
m.model_dump()

{'number': 0.33,
 'dt': datetime.datetime(2025, 1, 18, 23, 45, 26, 802005, tzinfo=datetime.timezone.utc)}

In [1939]:
m.model_dump_json()

'{"number":0.33,"dt":"2025-01-18"}'

Custom Validators